In [1]:
from numpy import *
import matplotlib.pyplot as plt
import operator
import time

In [2]:
'''
    def loadDataSet(fileName):从数据集文件fileName中读入数据，返回数据列表dataMat和标签列表labelMat
'''
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('   ')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

'''
    def selectJrand(i,m):从[1,m)中，随机返回一个不等于i的数
'''
def selectJrand(i,m):
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(random.uniform(0,m))
    return j

'''
    def clipAlpha(aj,H,L): 这个是剪辑函数，如果aj大于边界H，则返回H；如果aj小于边界L，则返回L；如果L<=aj<=H，则直接返回aj
'''
def clipAlpha(aj,H,L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj

'''
    def smoSimple(dataMatIn, classLabels, C, toler, maxIter):简化版的SMO算法。
    dataMatIn是数据列表，classLabels是标签列表，toler是松弛因子，C是toler的惩罚系数，maxIter是最大迭代次数。
    该函数首先遍历每个ai，然后再寻找当前ai对应合适的aj。如果遍历所有的ai都没有找到合适的aj，则重新遍历ai，但是该过程的最大次数为maxIter。
    该函数返回列表alphas、最佳的b
'''
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn);
    labelMat = mat(classLabels).transpose() #将classLabels从行向量转化为列向量
    b = 0; m,n = shape(dataMatrix)
    alphas = mat(zeros((m,1)))
    iter = 0 #iter表示寻找ai-aj对的连续失败次数
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m): #这是外循环，用于遍历所有的参数ai
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
                 # multiply(alphas,labelMat).T表示列向量alphas与列向量labelMat对应元素相乘，然后再转置为行向量
            Ei = fXi - float(labelMat[i])
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                 # ‘if’检查是否违反KKT条件,如果没有违反则继续选择第二个参数aj
                j = selectJrand(i,m)  #随机从区间[i,m)中选出一个aj
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print ("L==H"); continue #重新选择ai
                eta = dataMatrix[i,:]*dataMatrix[i,:].T + dataMatrix[j,:]*dataMatrix[j,:].T - 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T
                if eta <= 0: print ("eta<=0"); continue #eta<=0不能符号条件，进行下一轮,重新选择ai
                alphas[j] += labelMat[j]*(Ei - Ej)/eta #计算第二个参数aj
                alphas[j] = clipAlpha(alphas[j],H,L)
                if (abs(alphas[j] - alphaJold) < 0.00001): print ("j not moving enough"); continue
                alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j]) #通过aj和ai的关系，由aj求出ai
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2)/2.0
                alphaPairsChanged += 1 #ai与aj已经求出，alphaPairsChanged += 1
                print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
        if (alphaPairsChanged == 0): iter += 1 #如果遍历所有的ai都没有寻找到合适的aj，则表示寻找ai-aj对的连续失败次数iter+=1
        else: iter = 0 #找到了合适的ai-aj对，iter重置为0
        print ("iteration number: %d" % iter)
    return b,alphas


In [3]:
dataMatIn,classLabels=loadDataSet('testSet.txt')
b,alphas=smoSimple(dataMatIn,classLabels,0.6,0.001,40)
print(b,alphas[alphas>0])

iter: 0 i:0, pairs changed 1
j not moving enough
L==H
L==H
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
iter: 0 i:76, pairs changed 2
L==H
L==H
j not moving enough
iteration number: 0
j not moving enough
L==H
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
L==H
iter: 1 i:52, pairs changed 1
L==H
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
iter: 2 i:17, pairs changed 1
L==H
L==H
L

KeyboardInterrupt: 

In [4]:
dataMatIn
classLabels
C = 0.6
toler = 0.001
maxIter = 40

In [5]:
dataMatrix = mat(dataMatIn);
labelMat = mat(classLabels).transpose() #将classLabels从行向量转化为列向量
b = 0; m,n = shape(dataMatrix)
alphas = mat(zeros((m,1)))
iter = 0 #iter表示寻找ai-aj对的连续失败次数


In [ ]:
while (iter < maxIter):


In [6]:
alphaPairsChanged = 0



In [8]:
for i in range(m): #这是外循环，用于遍历所有的参数ai
    break
    


In [9]:
fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
     # multiply(alphas,labelMat).T表示列向量alphas与列向量labelMat对应元素相乘，然后再转置为行向量
Ei = fXi - float(labelMat[i])

In [10]:
fXi

0.0

In [11]:
Ei

1.0

In [12]:
((labelMat[i]*Ei < -toler) and (alphas[i] < C))

matrix([[ True]])

In [13]:
((labelMat[i]*Ei > toler) and (alphas[i] > 0))

matrix([[False]])

In [14]:
labelMat[i]

matrix([[-1.]])

In [15]:
-toler

-0.001

In [16]:
alphas[i] < C

matrix([[ True]])

In [17]:
C

0.6

In [ ]:

if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
    
    # ‘if’检查是否违反KKT条件,如果没有违反则继续选择第二个参数aj
    j = selectJrand(i,m)  #随机从区间[i,m)中选出一个aj
    fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
    Ej = fXj - float(labelMat[j])
    alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
    
    if (labelMat[i] != labelMat[j]):
        L = max(0, alphas[j] - alphas[i])
        H = min(C, C + alphas[j] - alphas[i])
    else:
        L = max(0, alphas[j] + alphas[i] - C)
        H = min(C, alphas[j] + alphas[i])
    
    if L==H: 
        print ("L==H"); 
        continue #重新选择ai
    eta = dataMatrix[i,:]*dataMatrix[i,:].T + dataMatrix[j,:]*dataMatrix[j,:].T - 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T
    if eta <= 0: 
        print ("eta<=0"); 
        continue #eta<=0不能符号条件，进行下一轮,重新选择ai
    
    alphas[j] += labelMat[j]*(Ei - Ej)/eta #计算第二个参数aj
    alphas[j] = clipAlpha(alphas[j],H,L)
    
    if (abs(alphas[j] - alphaJold) < 0.00001): 
        print ("j not moving enough"); 
        continue
        
    alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j]) #通过aj和ai的关系，由aj求出ai
    b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
    b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
    
    if (0 < alphas[i]) and (C > alphas[i]): 
        b = b1
    elif (0 < alphas[j]) and (C > alphas[j]): 
        b = b2
    else: b = (b1 + b2)/2.0
        
    alphaPairsChanged += 1 #ai与aj已经求出，alphaPairsChanged += 1
    print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))

In [ ]:
if (alphaPairsChanged == 0): iter += 1 #如果遍历所有的ai都没有寻找到合适的aj，则表示寻找ai-aj对的连续失败次数iter+=1
else: iter = 0 #找到了合适的ai-aj对，iter重置为0
print ("iteration number: %d" % iter)